In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv1D, Input
from tensorflow.keras.optimizers import RMSprop 
from tensorflow.keras import Model, utils
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.backend import clear_session
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.callbacks import ReduceLROnPlateau

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import os

%matplotlib inline

In [ ]:
def Load_data():
    '''
    Загружаем и соединяем данные.
    '''
    
    l_data1 = pd.read_csv('../input/udt-3-timeseries/18_19.csv', sep=';')
    l_data2 = pd.read_csv('../input/udt-3-timeseries/16_17.csv', sep=';')
    xTest = np.load('../input/udt-3-timeseries/x_test.npy')
    # Объединяем базы из двух файлов.
    lukoil_data = l_data2.append(l_data1)
    lukoil_data.drop(columns=['DATE', 'TIME'], axis=1, inplace=True)
    return lukoil_data, xTest

In [ ]:
# Считаем  базу данных.
lukoil_data, xTest = Load_data() 
lukoil_data.head()

In [ ]:
def Normalization_data(lukoil_data):
    '''
    Нормализация данных.
    '''
    
    global xScaler
    global yScaler
    global xLen
    global valLen

    xLen = 300 # Анализируем по xLen прошедшим точкам.
    valLen = 30000 # Используем valLen записей для проверки.
    
    lukoil_data = np.array(lukoil_data)
    trainLen = lukoil_data.shape[0]-valLen # Размер обучающей выборки.
     # Делим данные на обучающую и валидационную выборки.
    xTrain, xTest = lukoil_data[:trainLen], lukoil_data[trainLen+2:]
    # Масштабируем данные (отдельно для X и Y), чтобы их легче было скормить сетке.
    xScaler = MinMaxScaler()
    xScaler.fit(xTrain)
    xTrain = xScaler.transform(xTrain)
    xTest = xScaler.transform(xTest)
    # Сделаем reshape,т.к. у нас только один столбец по одному значению.
    yTrain = np.reshape(lukoil_data[:trainLen,3],(-1,1))
    yTest = np.reshape(lukoil_data[trainLen+2:,3],(-1,1)) 
    yScaler = MinMaxScaler()
    yScaler.fit(yTrain)
    yTrain = yScaler.transform(yTrain)
    yTest = yScaler.transform(yTest)
    print('Данные нормализованы успешно!')
    return xTrain, xTest, yTrain, yTest

In [ ]:
def Timeseries_data(xyTrain, parametr = 25):
    '''
    Метод для выборок с помощью генератора.
    '''
    
    xTrain = xyTrain[0]
    xTest = xyTrain[1]
    yTrain = xyTrain[2]
    yTest = xyTrain[3]
    
    # С помощью генератора создаем выборку для обучения.
    trainDataGen = TimeseriesGenerator(xTrain, yTrain,     # Параметров нашей выборки.
                                length=xLen, stride=1,     # Для каждой точки (из промежутка длины xLen).
                                batch_size=parametr)       # Размер batch, который будем подавать для модели.
    # Аналогичный генератор создадим для валидации при обучении.
    testDataGen = TimeseriesGenerator(xTest, yTest,
                                length=xLen, stride=1,
                                batch_size=parametr)
    print('Данные успешно сформированы!')
    return trainDataGen, testDataGen

In [ ]:
def plot_train_history(history, title): 
    '''
    Функция отрисовки графиков ошибки.
    '''
    
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(loss))

    plt.figure()
    plt.plot(epochs, loss, 'b', label='Ошибка на этапе обучения.')
    plt.plot(epochs, val_loss, 'r', label='Ошибка на этапе проверки.')
    plt.title(title)
    plt.legend()

    plt.show()

In [ ]:
# Нормализуем и создаём генератор для тренировачной и валидационной выборок.
Data_normal = Normalization_data(lukoil_data)
trainDataGen, testDataGen = Timeseries_data(Data_normal)

In [ ]:
print(trainDataGen[0][0].shape)
print(trainDataGen[0][1].shape)

In [ ]:
reduce_lr = ReduceLROnPlateau(min_lr=0.000000001, patience=3, verbose=0)

In [ ]:
clear_session()

In [ ]:
# Создаём модель.
Inputs = Input(shape=(300, 5))
x = Conv1D(300, 5, input_shape = (xLen,5), activation="linear")(Inputs)
x = Conv1D(100, 5, input_shape = (xLen,5), activation="linear")(x)
x = Flatten()(x)
x = Dense(100, activation="relu")(x)
outputs = Dense(1)(x)

model = Model(inputs=Inputs, outputs=outputs)
model.compile(optimizer=RMSprop(learning_rate=1e-4), loss="mse")

model.summary()

In [ ]:
# Тренируем модель.
history = model.fit(trainDataGen,
                    epochs=20,
                    batch_size=25,
                    validation_data=testDataGen,
                    callbacks=[reduce_lr],
                    verbose=1)

In [ ]:
plot_train_history(history, 'Потери на этапах обучения и проверки модели.')

In [ ]:
# Получим прогноз на тестовой выбоке.
def Predict_Model(xTest,currModel):
    '''
    Предсказываем ответ сети по тестовой выборке и возвращаем исходные масштаб данных, до нормализации.
    '''
    predTest = yScaler.inverse_transform(currModel.predict(xTest[0]))
    print('Созданы спрогнозированные значения, на части выделенных тестовых данных.')
    return predTest

In [ ]:
prediction_data = Predict_Model(xTest, model) # Прогноз на тестовой выбоке.

In [ ]:
# Оформление результата.
predUnscaled = prediction_data.squeeze()
# Создание датафрейма в нужном формате.
submission = pd.DataFrame({"Id":range(1,len(predUnscaled)+1),"Label":predUnscaled})
submission.head()
# Сохранение датафрейма как csv.
submission.to_csv('./SubmissionLukoilPrice.csv', sep=',', index=False, header=True)
